In [1]:
# importing required modules 
import pandas as pd
import re

from PyPDF2 import PdfReader

In [2]:
# Patterns to extract the different parts of the document
pattern = [re.compile('[The\s]?complaint\n?')]
pattern2 =[ re.compile(r'What happened'), re.compile(r'our initial conclusions')]
pattern3 = [re.compile('What I’ve decided – and why'), re.compile('my final decision')]

## Functions

In [3]:
def read_pdf(reader) -> list:
    #Extract all pdf pages
    info = []
    for page_number in range(reader.numPages):
        info.append(reader.getPage(page_number).extract_text())
    return info

In [4]:
def defective_pattern(text:str, pattern:re.Pattern) -> bool:
    if len(text) < 2:
        print(f'The pattern "{pattern}" is not working properly')
        return True
    return False

In [5]:
def get_part(pattern:list, plain_text:str, index:int):
    for idx, p in enumerate(pattern):
        text = re.split(pattern[idx], plain_text, maxsplit=1)
        if len(text) >= 2:
            break
    if defective_pattern(text, pattern[idx]):
        return None
    text = text[index]
    return text

In [6]:
# Split the document into different parts, and extract the two parts that we want
def extract_parts(plain_text:str) -> pd.DataFrame:
    text = get_part(pattern, plain_text, 1)
    
    # Extract first part
    first_part = get_part(pattern2, text, 0)

    # Extract second part
    second_text = get_part(pattern2, text, -1)
    second_part = get_part(pattern3, second_text, 0)
    
    # Extract DRN
    drn = re.findall(r'DRN-?\d+',plain_text)[0]
    
    
    (first_part, second_part, drn)
    return pd.DataFrame([{
        'The complaint': first_part,
        'What happened': second_part,
        'DRN': drn}])

## Execution 

In [35]:
# file_path = "//mnt/c/Users/cesar/Downloads/Decision Reference DRN-3495666 - DRN-3495666.pdf"
file_path = "//mnt/c/Users/cesar/Downloads/temp_file.pdf"

In [41]:
reader = PdfReader(file_path)
pdf_text = read_pdf(reader)
# Joining the different parts into one 
plain_text = ' '.join(pdf_text)

In [42]:
df = extract_parts(plain_text)

In [ ]:
# Save data in an Excel file 
df.to_excel('your_file_name.xlsx', index=False)